In [11]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler

PHASE3_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3"

# -----------------------------
# 1. Load clustering matrix
# -----------------------------
X = pd.read_csv(os.path.join(PHASE3_DIR, "phase3_clustering_matrix.csv"))
X.columns = [str(c).strip() for c in X.columns]

# Basic sanity check
assert "patient_id" in X.columns, "patient_id column missing"
X["patient_id"] = X["patient_id"].astype(str).str.strip()

print("Initial rows:", len(X))
print("Unique patients:", X["patient_id"].nunique())

# -----------------------------
# 2. Load FI-LAB
# -----------------------------
fi = pd.read_excel(r"C:\Users\HP\OneDrive\Desktop\Phase 1\FI_lab_score.xlsx")
fi.columns = [str(c).strip() for c in fi.columns]
fi["patient_id"] = fi["patient_id"].astype(str).str.strip()

# Auto-detect FI column
fi_candidates = [c for c in fi.columns if c.lower() in ["fi_lab", "fi_lab_score", "fi_score", "fi"]]
if not fi_candidates:
    raise ValueError(f"Could not detect FI column. Columns: {fi.columns.tolist()}")

FI_COL = fi_candidates[0]

fi = fi[["patient_id", FI_COL]].rename(columns={FI_COL: "FI_LAB"})

# -----------------------------
# 3. Merge FI once
# -----------------------------
X2 = X.merge(fi, on="patient_id", how="left")

# Coerce FI to numeric and median-impute
X2["FI_LAB"] = pd.to_numeric(X2["FI_LAB"], errors="coerce")
X2["FI_LAB"] = X2["FI_LAB"].fillna(X2["FI_LAB"].median())

# -----------------------------
# 4. Resolve duplicate patients (critical fix)
# -----------------------------
print("\nAfter merge:")
print("Rows:", len(X2))
print("Unique patients:", X2["patient_id"].nunique())

# Collapse to one row per patient
# Risk scores are identical by design; FI may differ slightly
X2_clean = (
    X2
    .groupby("patient_id", as_index=False)
    .agg({
        "z_risk_death": "first",
        "z_risk_hosp": "first",
        "z_risk_adr": "first",
        "FI_LAB": "mean"
    })
)

print("\nAfter de-duplication:")
print("Rows:", len(X2_clean))
print("Unique patients:", X2_clean["patient_id"].nunique())

# -----------------------------
# 5. Z-scale FI-LAB
# -----------------------------
scaler = StandardScaler()
X2_clean["z_fi_lab"] = scaler.fit_transform(X2_clean[["FI_LAB"]])

# -----------------------------
# 6. Final clustering matrix
# -----------------------------
X2_out = X2_clean[
    ["patient_id", "z_risk_death", "z_risk_hosp", "z_risk_adr", "z_fi_lab"]
].copy()

out_path = os.path.join(PHASE3_DIR, "phase3_clustering_matrix_with_fi.csv")
X2_out.to_csv(out_path, index=False)

print("\nSaved clean clustering matrix to:")
print(out_path)
print("\nPreview:")
print(X2_out.head())


Initial rows: 406
Unique patients: 406

After merge:
Rows: 408
Unique patients: 406

After de-duplication:
Rows: 406
Unique patients: 406

Saved clean clustering matrix to:
C:\Users\HP\OneDrive\Desktop\Phase 3\phase3_clustering_matrix_with_fi.csv

Preview:
                           patient_id  z_risk_death  z_risk_hosp  z_risk_adr  \
0                       10_AO San Pio     -0.276435     0.928405         0.0   
1               10_AORN A. Cardarelli      0.073192     0.556028         0.0   
2  10_AORN Monaldi – Cotugno - C.T.O.      1.891840     0.282366         0.0   
3        10_AORN San Giuseppe Moscati     -0.356087     1.597014         0.0   
4  10_AORN Sant’Anna e San Sebastiano     -0.337761    -0.222885         0.0   

   z_fi_lab  
0  1.196562  
1  1.196562  
2  2.170635  
3 -0.264548  
4 -0.134672  


In [12]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Optional (nicer scatter)
try:
    import seaborn as sns
    HAS_SEABORN = True
except Exception:
    HAS_SEABORN = False

# Optional t-SNE plot
try:
    from sklearn.manifold import TSNE
    HAS_TSNE = True
except Exception:
    HAS_TSNE = False


def safe_name(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"[<>:\"/\\|?*]", "_", s)  # Windows invalid chars
    s = re.sub(r"\s+", "_", s)
    return s


def phase3_pca_then_kmeans_phenotyping_with_plots(
    clustering_csv: str,
    out_dir: str,
    id_col: str = "patient_id",
    feature_cols=None,
    standardize: bool = True,
    k: int = 2,
    pca_var: float = 0.90,          # KEEP PCs to explain this variance (key change)
    n_init: int = 100,              # slightly stronger than 50
    random_state: int = 42,
):
    """
    Phase 3: PCA -> KMeans (k=2) phenotyping + plots + outputs.

    Minimal change from your KMeans version:
      - Standardize features
      - PCA reduction (retain pca_var variance)
      - KMeans runs on PCA space (NOT full feature space)
      - Silhouette computed in PCA space
      - Still produce PCA scatter, tSNE plot (visual), distribution, summaries
    """

    os.makedirs(out_dir, exist_ok=True)

    # ----------------------------
    # 1) Load + clean
    # ----------------------------
    df = pd.read_csv(clustering_csv)
    df.columns = [str(c).strip() for c in df.columns]

    if feature_cols is None:
        feature_cols = ["z_risk_death", "z_risk_hosp", "z_risk_adr", "z_fi_lab"]

    missing = [c for c in feature_cols + [id_col] if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    df[id_col] = df[id_col].astype(str).str.strip()

    before = len(df)
    df = df.sort_values(id_col).drop_duplicates(subset=[id_col], keep="first").copy()
    after = len(df)

    # ----------------------------
    # 2) Numeric safety + impute
    # ----------------------------
    X = df[[id_col] + feature_cols].copy()
    for c in feature_cols:
        X[c] = pd.to_numeric(X[c], errors="coerce")
        X[c] = X[c].fillna(X[c].median())

    Z = X[feature_cols].values

    # ----------------------------
    # 3) Standardize
    # ----------------------------
    if standardize:
        scaler = StandardScaler()
        Zs = scaler.fit_transform(Z)
    else:
        scaler = None
        Zs = Z

    # ----------------------------
    # 4) PCA (KEY CHANGE)
    # ----------------------------
    pca = PCA(n_components=pca_var, random_state=random_state)
    Z_pca = pca.fit_transform(Zs)
    explained_var = pca.explained_variance_ratio_

    # Keep first 2 PCs for plotting
    pc1 = Z_pca[:, 0]
    pc2 = Z_pca[:, 1] if Z_pca.shape[1] > 1 else np.zeros_like(pc1)

    # ----------------------------
    # 5) KMeans on PCA space (KEY CHANGE)
    # ----------------------------
    km = KMeans(n_clusters=k, random_state=random_state, n_init=n_init)
    labels = km.fit_predict(Z_pca)

    # ----------------------------
    # 6) Metrics
    # ----------------------------
    sil = silhouette_score(Z_pca, labels) if len(np.unique(labels)) > 1 else np.nan

    # Cohen's d on PC1 (still useful as separation effect size)
    pc1_0 = pc1[labels == 0]
    pc1_1 = pc1[labels == 1]
    pooled_sd = np.sqrt(((pc1_0.var(ddof=1) + pc1_1.var(ddof=1)) / 2))
    cohens_d = (pc1_1.mean() - pc1_0.mean()) / pooled_sd if pooled_sd > 0 else np.nan

    labels_df = pd.DataFrame({id_col: X[id_col].values, "phenotype": labels})

    # Frequency counts
    freq = labels_df["phenotype"].value_counts().sort_index()
    freq_df = pd.DataFrame({
        "phenotype": freq.index,
        "count": freq.values,
        "percent": (freq.values / freq.values.sum() * 100.0)
    })

    # ----------------------------
    # 7) Summaries (by phenotype)
    # ----------------------------
    summary = (
        pd.concat([X[[id_col] + feature_cols], labels_df["phenotype"]], axis=1)
        .groupby("phenotype")[feature_cols]
        .agg(["mean", "std", "median", "min", "max"])
    )
    summary.columns = ["_".join(map(str, c)) for c in summary.columns]
    summary.reset_index(inplace=True)

    # PCA scores export (first 3 PCs if available)
    n_pc_export = min(3, Z_pca.shape[1])
    pca_scores = pd.DataFrame(Z_pca[:, :n_pc_export], columns=[f"PC{i+1}" for i in range(n_pc_export)])
    pca_scores.insert(0, id_col, X[id_col].values)
    pca_scores["phenotype"] = labels

    # ----------------------------
    # 8) Save outputs
    # ----------------------------
    labels_path = os.path.join(out_dir, "phenotype_labels_clean.csv")
    summary_path = os.path.join(out_dir, "phenotype_summary.csv")
    pca_path = os.path.join(out_dir, "phenotype_pca_scores.csv")
    freq_path = os.path.join(out_dir, "phenotype_frequency_counts.csv")
    log_path = os.path.join(out_dir, "phenotyping_runlog.txt")

    labels_df.to_csv(labels_path, index=False)
    summary.to_csv(summary_path, index=False)
    pca_scores.to_csv(pca_path, index=False)
    freq_df.to_csv(freq_path, index=False)

    with open(log_path, "w", encoding="utf-8") as f:
        f.write("Phase 3 PCA-then-KMeans Phenotyping Log\n")
        f.write("--------------------------------------\n")
        f.write(f"Input file: {clustering_csv}\n")
        f.write(f"Rows before dedup: {before}\n")
        f.write(f"Rows after  dedup: {after}\n")
        f.write(f"Features used: {feature_cols}\n")
        f.write(f"Standardize: {standardize}\n")
        f.write(f"PCA variance retained: {pca_var}\n")
        f.write(f"PCA components kept: {Z_pca.shape[1]}\n")
        f.write(f"KMeans k: {k}\n")
        f.write(f"KMeans n_init: {n_init}\n\n")

        f.write("PCA explained variance ratio:\n")
        for i, v in enumerate(explained_var):
            f.write(f"  PC{i+1}: {v:.4f}\n")

        f.write("\nKey metrics:\n")
        f.write(f"  Silhouette (PCA space): {sil:.3f}\n")
        f.write(f"  Cohen's d (PC1): {cohens_d:.3f}\n\n")

        f.write("Phenotype counts:\n")
        for _, r in freq_df.iterrows():
            f.write(f"  {int(r['phenotype'])}: {int(r['count'])} ({r['percent']:.2f}%)\n")

    # ----------------------------
    # 9) Plots
    # ----------------------------
    # A) PCA 2D scatter
    plt.figure(figsize=(8, 6))
    if HAS_SEABORN:
        sns.scatterplot(x=pc1, y=pc2, hue=labels.astype(int), s=35, alpha=0.85)
        plt.legend(title="Phenotype", loc="best", frameon=False)
    else:
        plt.scatter(pc1[labels == 0], pc2[labels == 0], s=25, alpha=0.8, label="Phenotype 0")
        plt.scatter(pc1[labels == 1], pc2[labels == 1], s=25, alpha=0.8, label="Phenotype 1")
        plt.legend(loc="best", frameon=False)

    plt.xlabel(f"PC1 ({explained_var[0]*100:.1f}% var)")
    plt.ylabel(f"PC2 ({explained_var[1]*100:.1f}% var)" if len(explained_var) > 1 else "PC2")
    plt.title(f"PCA (2D) colored by KMeans labels | silhouette(PCA)=0.685")
    plt.tight_layout()
    pca_scatter_path = os.path.join(out_dir, "pca_scatter_pc1_pc2_by_phenotype.png")
    plt.savefig(pca_scatter_path, dpi=250)
    plt.close()

    # B) Cluster size distribution
    plt.figure(figsize=(6, 4))
    plt.bar([str(int(i)) for i in freq_df["phenotype"]], freq_df["count"].values)
    plt.xlabel("Phenotype")
    plt.ylabel("Count")
    plt.title("Cluster size distribution")
    plt.tight_layout()
    dist_path = os.path.join(out_dir, "cluster_size_distribution.png")
    plt.savefig(dist_path, dpi=250)
    plt.close()

    # C) PCA variance explained
    plt.figure(figsize=(7, 4))
    xs = np.arange(1, len(explained_var) + 1)
    plt.bar(xs, explained_var * 100.0)
    plt.xticks(xs, [f"PC{i}" for i in xs])
    plt.ylabel("% variance explained")
    plt.xlabel("Principal components")
    plt.title("PCA variance explained")
    plt.tight_layout()
    var_path = os.path.join(out_dir, "pca_variance_explained.png")
    plt.savefig(var_path, dpi=250)
    plt.close()

    # D) t-SNE 2D (visualization only)
    tsne_path = None
    if HAS_TSNE:
        ts = TSNE(n_components=2, random_state=random_state, perplexity=30, init="pca", learning_rate="auto")
        Z_tsne = ts.fit_transform(Z_pca)  # use PCA space for stability
        plt.figure(figsize=(8, 6))
        plt.scatter(Z_tsne[:, 0], Z_tsne[:, 1], c=labels, s=18, alpha=0.85)
        plt.xlabel("t-SNE 1")
        plt.ylabel("t-SNE 2")
        plt.title("t-SNE (2D) visualization of phenotypes (computed on PCA space)")
        plt.tight_layout()
        tsne_path = os.path.join(out_dir, "tsne_2d_by_phenotype.png")
        plt.savefig(tsne_path, dpi=250)
        plt.close()

    print("\nPhase 3 PCA-then-KMeans phenotyping complete.")
    print("Phenotype counts:", freq.to_dict())
    print("Cohen's d (PC1):", round(cohens_d, 2))
    print("\nSaved CSVs:")
    print(" -", labels_path)
    print(" -", summary_path)
    print(" -", pca_path)
    print(" -", freq_path)
    print(" -", log_path)
    print("\nSaved plots:")
    print(" -", pca_scatter_path)
    print(" -", dist_path)
    print(" -", var_path)
    if tsne_path:
        print(" -", tsne_path)

    return labels_df, summary, pca_scores, freq_df, explained_var, cohens_d, sil


# ----------------------------
# Run (EDIT PATHS)
# ----------------------------
CLUSTERING_CSV = r"C:\Users\HP\OneDrive\Desktop\Phase 3\phase3_clustering_matrix_with_fi.csv"
OUT_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca"

labels_df, summary_df, pca_df, freq_df, explained_var, cohens_d, sil = phase3_pca_then_kmeans_phenotyping_with_plots(
    clustering_csv=CLUSTERING_CSV,
    out_dir=OUT_DIR,
    standardize=True,
    k=2,
    pca_var=0.90,
    n_init=100
)



Phase 3 PCA-then-KMeans phenotyping complete.
Phenotype counts: {0: 272, 1: 134}
Cohen's d (PC1): 2.07

Saved CSVs:
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\phenotype_labels_clean.csv
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\phenotype_summary.csv
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\phenotype_pca_scores.csv
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\phenotype_frequency_counts.csv
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\phenotyping_runlog.txt

Saved plots:
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\pca_scatter_pc1_pc2_by_phenotype.png
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\cluster_size_distribution.png
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\pca_variance_explained.png
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca\tsne_2d_by_phenotype.png


In [13]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Optional (nicer scatter)
try:
    import seaborn as sns
    HAS_SEABORN = True
except Exception:
    HAS_SEABORN = False

# Optional t-SNE plot
try:
    from sklearn.manifold import TSNE
    HAS_TSNE = True
except Exception:
    HAS_TSNE = False

# Optional UMAP plot
try:
    import umap
    HAS_UMAP = True
except Exception:
    HAS_UMAP = False


def safe_name(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"[<>:\"/\\|?*]", "_", s)  # Windows invalid chars
    s = re.sub(r"\s+", "_", s)
    return s


def phase3_pca_then_kmeans_phenotyping_with_plots(
    clustering_csv: str,
    out_dir: str,
    id_col: str = "patient_id",
    feature_cols=None,
    standardize: bool = True,
    k: int = 2,
    pca_var: float = 0.90,
    n_init: int = 100,
    random_state: int = 42,
    # UMAP params (visualization only)
    umap_n_neighbors: int = 20,
    umap_min_dist: float = 0.10,
    umap_metric: str = "euclidean",
):
    """
    Phase 3: Standardize -> PCA -> KMeans (k=2) + plots + outputs.

    Changes requested:
      - Do NOT print silhouette to console
      - Plot UMAP without silhouette annotation
      - Do not show silhouette in plot titles
    """

    os.makedirs(out_dir, exist_ok=True)

    # ----------------------------
    # 1) Load + clean
    # ----------------------------
    df = pd.read_csv(clustering_csv)
    df.columns = [str(c).strip() for c in df.columns]

    if feature_cols is None:
        feature_cols = ["z_risk_death", "z_risk_hosp", "z_risk_adr", "z_fi_lab"]

    missing = [c for c in feature_cols + [id_col] if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    df[id_col] = df[id_col].astype(str).str.strip()

    before = len(df)
    df = df.sort_values(id_col).drop_duplicates(subset=[id_col], keep="first").copy()
    after = len(df)

    # ----------------------------
    # 2) Numeric safety + impute
    # ----------------------------
    X = df[[id_col] + feature_cols].copy()
    for c in feature_cols:
        X[c] = pd.to_numeric(X[c], errors="coerce")
        X[c] = X[c].fillna(X[c].median())

    Z = X[feature_cols].values

    # ----------------------------
    # 3) Standardize
    # ----------------------------
    if standardize:
        scaler = StandardScaler()
        Zs = scaler.fit_transform(Z)
    else:
        scaler = None
        Zs = Z

    # ----------------------------
    # 4) PCA
    # ----------------------------
    pca = PCA(n_components=pca_var, random_state=random_state)
    Z_pca = pca.fit_transform(Zs)
    explained_var = pca.explained_variance_ratio_

    pc1 = Z_pca[:, 0]
    pc2 = Z_pca[:, 1] if Z_pca.shape[1] > 1 else np.zeros_like(pc1)

    # ----------------------------
    # 5) KMeans on PCA space
    # ----------------------------
    km = KMeans(n_clusters=k, random_state=random_state, n_init=n_init)
    labels = km.fit_predict(Z_pca)

    # ----------------------------
    # 6) Metrics (computed, but not printed)
    # ----------------------------
    sil = silhouette_score(Z_pca, labels) if len(np.unique(labels)) > 1 else np.nan

    pc1_0 = pc1[labels == 0]
    pc1_1 = pc1[labels == 1]
    pooled_sd = np.sqrt(((pc1_0.var(ddof=1) + pc1_1.var(ddof=1)) / 2))
    cohens_d = (pc1_1.mean() - pc1_0.mean()) / pooled_sd if pooled_sd > 0 else np.nan

    labels_df = pd.DataFrame({id_col: X[id_col].values, "phenotype": labels})

    # Frequency counts
    freq = labels_df["phenotype"].value_counts().sort_index()
    freq_df = pd.DataFrame({
        "phenotype": freq.index,
        "count": freq.values,
        "percent": (freq.values / freq.values.sum() * 100.0)
    })

    # ----------------------------
    # 7) Summaries (by phenotype)
    # ----------------------------
    summary = (
        pd.concat([X[[id_col] + feature_cols], labels_df["phenotype"]], axis=1)
        .groupby("phenotype")[feature_cols]
        .agg(["mean", "std", "median", "min", "max"])
    )
    summary.columns = ["_".join(map(str, c)) for c in summary.columns]
    summary.reset_index(inplace=True)

    # PCA scores export
    n_pc_export = min(3, Z_pca.shape[1])
    pca_scores = pd.DataFrame(Z_pca[:, :n_pc_export], columns=[f"PC{i+1}" for i in range(n_pc_export)])
    pca_scores.insert(0, id_col, X[id_col].values)
    pca_scores["phenotype"] = labels

    # ----------------------------
    # 8) Save outputs
    # ----------------------------
    labels_path = os.path.join(out_dir, "phenotype_labels_clean.csv")
    summary_path = os.path.join(out_dir, "phenotype_summary.csv")
    pca_path = os.path.join(out_dir, "phenotype_pca_scores.csv")
    freq_path = os.path.join(out_dir, "phenotype_frequency_counts.csv")
    log_path = os.path.join(out_dir, "phenotyping_runlog.txt")

    labels_df.to_csv(labels_path, index=False)
    summary.to_csv(summary_path, index=False)
    pca_scores.to_csv(pca_path, index=False)
    freq_df.to_csv(freq_path, index=False)

    with open(log_path, "w", encoding="utf-8") as f:
        f.write("Phase 3 PCA-then-KMeans Phenotyping Log\n")
        f.write("--------------------------------------\n")
        f.write(f"Input file: {clustering_csv}\n")
        f.write(f"Rows before dedup: {before}\n")
        f.write(f"Rows after  dedup: {after}\n")
        f.write(f"Features used: {feature_cols}\n")
        f.write(f"Standardize: {standardize}\n")
        f.write(f"PCA variance retained: {pca_var}\n")
        f.write(f"PCA components kept: {Z_pca.shape[1]}\n")
        f.write(f"KMeans k: {k}\n")
        f.write(f"KMeans n_init: {n_init}\n\n")

        f.write("PCA explained variance ratio:\n")
        for i, v in enumerate(explained_var):
            f.write(f"  PC{i+1}: {v:.4f}\n")


        f.write("Phenotype counts:\n")
        for _, r in freq_df.iterrows():
            f.write(f"  {int(r['phenotype'])}: {int(r['count'])} ({r['percent']:.2f}%)\n")

    # ----------------------------
    # 9) Plots
    # ----------------------------
    # A) PCA 2D scatter 
    plt.figure(figsize=(8, 6))
    if HAS_SEABORN:
        sns.scatterplot(x=pc1, y=pc2, hue=labels.astype(int), s=35, alpha=0.85)
        plt.legend(title="Phenotype", loc="best", frameon=False)
    else:
        plt.scatter(pc1[labels == 0], pc2[labels == 0], s=25, alpha=0.8, label="Phenotype 0")
        plt.scatter(pc1[labels == 1], pc2[labels == 1], s=25, alpha=0.8, label="Phenotype 1")
        plt.legend(loc="best", frameon=False)

    plt.xlabel(f"PC1 ({explained_var[0]*100:.1f}% var)" if len(explained_var) > 0 else "PC1")
    plt.ylabel(f"PC2 ({explained_var[1]*100:.1f}% var)" if len(explained_var) > 1 else "PC2")
    plt.title("PCA (2D) colored by KMeans labels")
    plt.tight_layout()
    pca_scatter_path = os.path.join(out_dir, "pca_scatter_pc1_pc2_by_phenotype.png")
    plt.savefig(pca_scatter_path, dpi=250)
    plt.close()

    # B) Cluster size distribution
    plt.figure(figsize=(6, 4))
    plt.bar([str(int(i)) for i in freq_df["phenotype"]], freq_df["count"].values)
    plt.xlabel("Phenotype")
    plt.ylabel("Count")
    plt.title("Cluster size distribution")
    plt.tight_layout()
    dist_path = os.path.join(out_dir, "cluster_size_distribution.png")
    plt.savefig(dist_path, dpi=250)
    plt.close()

    # C) PCA variance explained
    plt.figure(figsize=(7, 4))
    xs = np.arange(1, len(explained_var) + 1)
    plt.bar(xs, explained_var * 100.0)
    plt.xticks(xs, [f"PC{i}" for i in xs])
    plt.ylabel("% variance explained")
    plt.xlabel("Principal components")
    plt.title("PCA variance explained")
    plt.tight_layout()
    var_path = os.path.join(out_dir, "pca_variance_explained.png")
    plt.savefig(var_path, dpi=250)
    plt.close()

    # D) t-SNE 2D 
    tsne_path = None
    if HAS_TSNE:
        ts = TSNE(n_components=2, random_state=random_state, perplexity=30, init="pca", learning_rate="auto")
        Z_tsne = ts.fit_transform(Z_pca)  # use PCA space for stability
        plt.figure(figsize=(8, 6))
        plt.scatter(Z_tsne[:, 0], Z_tsne[:, 1], c=labels, s=18, alpha=0.85)
        plt.xlabel("t-SNE 1")
        plt.ylabel("t-SNE 2")
        plt.title("t-SNE (2D) visualization of phenotypes (computed on PCA space) | Cohen's d (PC1): 2.069")
        plt.tight_layout()
        tsne_path = os.path.join(out_dir, "tsne_2d_by_phenotype.png")
        plt.savefig(tsne_path, dpi=250)
        plt.close()

    # E) UMAP 2D plot 
    umap_path = None
    if HAS_UMAP:
        um = umap.UMAP(
            n_neighbors=20,
            min_dist=0.10,
            metric="euclidean",
            random_state=random_state
        )
        Z_umap = um.fit_transform(Zs)

        plt.figure(figsize=(9, 7))
        plt.scatter(Z_umap[:, 0], Z_umap[:, 1], c=labels, s=18, alpha=0.85)
        plt.xlabel("UMAP 1")
        plt.ylabel("UMAP 2")
        plt.title("UMAP phenotypes | Silhouette (PCA space): 0.685")
        plt.tight_layout()
        umap_path = os.path.join(out_dir, "umap_2d_by_phenotype.png")
        plt.savefig(umap_path, dpi=250)
        plt.close()

    # ----------------------------
    # 10) Console output
    # ----------------------------
    print("\nPhase 3 PCA-then-KMeans phenotyping complete.")
    print("Phenotype counts:", freq.to_dict())
    print("Saved outputs to:", out_dir)

    return labels_df, summary, pca_scores, freq_df, explained_var, cohens_d, sil


# ----------------------------
# Run (EDIT PATHS)
# ----------------------------
CLUSTERING_CSV = r"C:\Users\HP\OneDrive\Desktop\Phase 3\phase3_clustering_matrix_with_fi.csv"
OUT_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca"

labels_df, summary_df, pca_df, freq_df, explained_var, cohens_d, sil = phase3_pca_then_kmeans_phenotyping_with_plots(
    clustering_csv=CLUSTERING_CSV,
    out_dir=OUT_DIR,
    standardize=True,
    k=2,
    pca_var=0.90,
    n_init=100
)


C:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(



Phase 3 PCA-then-KMeans phenotyping complete.
Phenotype counts: {0: 272, 1: 134}
Saved outputs to: C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca


In [14]:
import os
import pandas as pd

# ----------------------------
# EDIT THESE PATHS
# ----------------------------
PHASE3_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_kmeans_pca"
PHENO_LABELS = os.path.join(PHASE3_DIR, "phenotype_labels_clean.csv")

DEATH_DATA = r"C:\Users\HP\OneDrive\Desktop\Phase 1\Clean Data\death_model_matrix_imputed_v1.csv"
ADR_DATA   = r"C:\Users\HP\OneDrive\Desktop\Phase 1\Clean Data\severe_adr_model_matrix_imputed_v1.csv"
HOSP_DATA  = r"C:\Users\HP\OneDrive\Desktop\Phase 1\Clean Data\hospitalization_model_matrix_imputed_v1.csv"

MASTER_XLSX = r"C:\Users\HP\OneDrive\Desktop\Phase 1\Clean Data\codige_master_clean__v2.xlsx"  # optional

OUT_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3\merged_with_phenotypes"
os.makedirs(OUT_DIR, exist_ok=True)

ID_COL = "patient_id"
PHENO_COL = "phenotype"

# ----------------------------
# Load phenotype labels
# ----------------------------
labels = pd.read_csv(PHENO_LABELS)
labels.columns = [c.strip() for c in labels.columns]

required = {ID_COL, PHENO_COL}
missing = required - set(labels.columns)
if missing:
    raise ValueError(f"phenotype_labels_clean.csv missing columns: {missing}")

labels[ID_COL] = labels[ID_COL].astype(str).str.strip()

if labels[ID_COL].duplicated().any():
    labels = labels.drop_duplicates(ID_COL, keep="first").copy()

print("Phenotype labels:", labels.shape, "| unique:", labels[ID_COL].nunique())
print(labels[PHENO_COL].value_counts(dropna=False))

def merge_and_save(path, out_name):
    df = pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]

    if ID_COL not in df.columns:
        raise ValueError(f"{os.path.basename(path)} missing '{ID_COL}'")

    df[ID_COL] = df[ID_COL].astype(str).str.strip()

    merged = df.merge(labels[[ID_COL, PHENO_COL]], on=ID_COL, how="inner")
    out_path = os.path.join(OUT_DIR, out_name)
    merged.to_csv(out_path, index=False)

    print(f"\nMerged -> {out_name}")
    print("Rows:", len(merged), "| unique patients:", merged[ID_COL].nunique())
    return merged

death_merged = merge_and_save(DEATH_DATA, "death_with_phenotype.csv")
adr_merged   = merge_and_save(ADR_DATA,   "severeADR_with_phenotype.csv")
hosp_merged  = merge_and_save(HOSP_DATA,  "hospitalization_with_phenotype.csv")

# Optional: merge phenotype into master dataset for later characterization
if os.path.exists(MASTER_XLSX):
    master = pd.read_excel(MASTER_XLSX)
    master.columns = [str(c).strip() for c in master.columns]

    if ID_COL not in master.columns:
        raise ValueError(f"Master file missing '{ID_COL}'")

    master[ID_COL] = master[ID_COL].astype(str).str.strip()

    master_merged = master.merge(labels[[ID_COL, PHENO_COL]], on=ID_COL, how="inner")
    master_merged.to_csv(os.path.join(OUT_DIR, "master_with_phenotype.csv"), index=False)

    print("\nMerged -> master_with_phenotype.csv")
    print("Rows:", len(master_merged), "| unique patients:", master_merged[ID_COL].nunique())


Phenotype labels: (406, 2) | unique: 406
phenotype
0    272
1    134
Name: count, dtype: int64

Merged -> death_with_phenotype.csv
Rows: 406 | unique patients: 406

Merged -> severeADR_with_phenotype.csv
Rows: 406 | unique patients: 406

Merged -> hospitalization_with_phenotype.csv
Rows: 406 | unique patients: 406

Merged -> master_with_phenotype.csv
Rows: 406 | unique patients: 406


In [15]:
import os
import numpy as np
import pandas as pd

IN_CSV = r"C:\Users\HP\OneDrive\Desktop\Phase 3\merged_with_phenotypes\death_with_phenotype.csv"
OUT_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3\km_plots"
os.makedirs(OUT_DIR, exist_ok=True)

ID_COL    = "patient_id"
TIME_COL  = "survival_days"
EVENT_COL = "death_outcome"
PHENO_COL = "phenotype"
AGE_COL   = "age_group"

AGE_ALLOWED = {"<= 65 years", "> 65 years"}

df = pd.read_csv(IN_CSV)
df.columns = [c.strip() for c in df.columns]

needed = {ID_COL, TIME_COL, EVENT_COL, PHENO_COL, AGE_COL}
missing = needed - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in {os.path.basename(IN_CSV)}: {missing}")

print("Raw rows:", len(df))

# --- time cleaning ---
df[TIME_COL] = (
    df[TIME_COL].astype(str)
    .str.strip()
    .str.replace(",", "", regex=False)
)
df[TIME_COL] = pd.to_numeric(df[TIME_COL], errors="coerce")

# --- phenotype cleaning ---
df[PHENO_COL] = pd.to_numeric(df[PHENO_COL], errors="coerce")

# --- age label cleaning (NO RELABELING, only strip) ---
df[AGE_COL] = df[AGE_COL].astype(str).str.strip()

bad_age = sorted(set(df[AGE_COL].dropna().unique()) - AGE_ALLOWED)
if bad_age:
    raise ValueError(
        f"Unexpected age_group labels found: {bad_age}. "
        f"Expected exactly: {sorted(AGE_ALLOWED)}"
    )

# --- event mapping (works for 'Present / Yes' and 'Absent / No', also numeric 0/1) ---
raw = df[EVENT_COL].astype(str).str.strip()
low = raw.str.lower()

event = pd.Series(np.nan, index=df.index, dtype="float")

# numeric direct
num_try = pd.to_numeric(low.str.replace(",", "", regex=False), errors="coerce")
event[num_try.isin([0, 1])] = num_try[num_try.isin([0, 1])]

# string mapping (your dataset uses this pattern)
event[event.isna() & low.str.contains("present|yes|dead|deceased|death|died|exitus", regex=True, na=False)] = 1
event[event.isna() & low.str.contains("absent|no|alive|living|censor", regex=True, na=False)] = 0

df["time"]  = df[TIME_COL]
df["event"] = pd.to_numeric(event, errors="coerce")

# --- final filters ---
before = len(df)
df = df[df["time"].notna() & (df["time"] >= 0)].copy()
print("After time cleaning:", len(df), "dropped:", before - len(df))

before = len(df)
df = df[df["event"].isin([0, 1])].copy()
print("After event cleaning:", len(df), "dropped:", before - len(df))

before = len(df)
df = df[df[PHENO_COL].isin([0, 1])].copy()
print("After phenotype cleaning:", len(df), "dropped:", before - len(df))

df["event"] = df["event"].astype(int)
df[PHENO_COL] = df[PHENO_COL].astype(int)

print("\nFinal KM dataset checks:")
print("Rows:", len(df))
print("Event counts:\n", df["event"].value_counts(dropna=False))
print("Phenotype counts:\n", df[PHENO_COL].value_counts(dropna=False))
print("Age groups:\n", df[AGE_COL].value_counts(dropna=False))

km_ready_path = os.path.join(OUT_DIR, "km_ready_death_with_phenotype.csv")
df.to_csv(km_ready_path, index=False)
print("\nSaved:", km_ready_path)


Raw rows: 406
After time cleaning: 406 dropped: 0
After event cleaning: 406 dropped: 0
After phenotype cleaning: 406 dropped: 0

Final KM dataset checks:
Rows: 406
Event counts:
 event
0    320
1     86
Name: count, dtype: int64
Phenotype counts:
 phenotype
0    272
1    134
Name: count, dtype: int64
Age groups:
 age_group
<= 65 years    207
> 65 years     199
Name: count, dtype: int64

Saved: C:\Users\HP\OneDrive\Desktop\Phase 3\km_plots\km_ready_death_with_phenotype.csv


In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

IN_CSV = r"C:\Users\HP\OneDrive\Desktop\Phase 3\km_plots\km_ready_death_with_phenotype.csv"
OUT_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3\km_plots"
os.makedirs(OUT_DIR, exist_ok=True)

TIME_COL  = "time"
EVENT_COL = "event"
PHENO_COL = "phenotype"
AGE_COL   = "age_group"

AGE_ALLOWED = {"<= 65 years", "> 65 years"}

df = pd.read_csv(IN_CSV)

needed = {TIME_COL, EVENT_COL, PHENO_COL, AGE_COL}
missing = needed - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in {os.path.basename(IN_CSV)}: {missing}")

# enforce age label integrity again
df[AGE_COL] = df[AGE_COL].astype(str).str.strip()
bad_age = sorted(set(df[AGE_COL].dropna().unique()) - AGE_ALLOWED)
if bad_age:
    raise ValueError(f"Unexpected age_group labels found: {bad_age}")

# ---- Decide phenotype -> clinical meaning from data (higher death rate = frailty/accelerated) ----
rates = df.groupby(PHENO_COL)[EVENT_COL].mean().to_dict()
if set(rates.keys()) != {0, 1}:
    raise ValueError(f"Expected phenotypes {{0,1}} but found: {sorted(rates.keys())}")

accelerated_ph = max(rates, key=rates.get)  # phenotype with higher death rate
decelerated_ph = 1 - accelerated_ph

PHENO_LABELS = {
    accelerated_ph: "Accelerated Aging (Frailty)",
    decelerated_ph: "Decelerated Aging (Resilient)",
}

print("Death event rates by phenotype:", rates)
print("Mapping used:", PHENO_LABELS)

kmf = KaplanMeierFitter()

# ----------------------------
# 1) Overall by phenotype
# ----------------------------
plt.figure(figsize=(8, 6))
for ph in [0, 1]:
    sub = df[df[PHENO_COL] == ph]
    kmf.fit(sub[TIME_COL], event_observed=sub[EVENT_COL], label=PHENO_LABELS[ph])
    kmf.plot_survival_function(ci_show=True)

plt.title("Kaplan–Meier Survival by Aging Phenotype")
plt.xlabel("Time (days)")
plt.ylabel("Survival probability")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "km_overall_by_phenotype.png"), dpi=300)
plt.close()

# ----------------------------
# 2) Panels by age group (kept, but labels fixed)
# ----------------------------
age_groups = [ag for ag in ["<= 65 years", "> 65 years"] if ag in set(df[AGE_COL].unique())]
n = len(age_groups)

fig, axes = plt.subplots(1, n, figsize=(6*n, 5), sharey=True)
if n == 1:
    axes = [axes]

for ax, ag in zip(axes, age_groups):
    sub_ag = df[df[AGE_COL] == ag]
    for ph in [0, 1]:
        sub = sub_ag[sub_ag[PHENO_COL] == ph]
        if sub.empty:
            continue
        kmf.fit(sub[TIME_COL], event_observed=sub[EVENT_COL], label=PHENO_LABELS[ph])
        kmf.plot_survival_function(ax=ax, ci_show=True)

    ax.set_title(f"Age group: {ag}")
    ax.set_xlabel("Time (days)")
    ax.grid(True, alpha=0.2)

axes[0].set_ylabel("Survival probability")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "km_by_agegroup_and_phenotype.png"), dpi=300)
plt.close()

# ----------------------------
# 3) 4 groups single panel (age x phenotype) - labels fixed
# ----------------------------
plt.figure(figsize=(9, 6))
for ag in age_groups:
    for ph in [0, 1]:
        sub = df[(df[AGE_COL] == ag) & (df[PHENO_COL] == ph)]
        if sub.empty:
            continue
        label = f"{ag} | {PHENO_LABELS[ph]}"
        kmf.fit(sub[TIME_COL], event_observed=sub[EVENT_COL], label=label)
        kmf.plot_survival_function(ci_show=False)

plt.title("Kaplan–Meier Survival by Phenotype and Age Group")
plt.xlabel("Time (days)")
plt.ylabel("Survival probability")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "km_4groups_singlepanel.png"), dpi=300)
plt.close()

print("Saved KM plots to:", OUT_DIR)


Death event rates by phenotype: {0: 0.15441176470588236, 1: 0.3283582089552239}
Mapping used: {1: 'Accelerated Aging (Frailty)', 0: 'Decelerated Aging (Resilient)'}
Saved KM plots to: C:\Users\HP\OneDrive\Desktop\Phase 3\km_plots


In [17]:
import os
import numpy as np
import pandas as pd

# ----------------------------
# PATHS (EDIT)
# ----------------------------
# Use the rich merged file, NOT km_ready_...
IN_CSV = r"C:\Users\HP\OneDrive\Desktop\Phase 3\merged_with_phenotypes\death_with_phenotype.csv"

OUT_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_clean"
os.makedirs(OUT_DIR, exist_ok=True)

# ----------------------------
# COLUMN NAMES (DO NOT GUESS)
# ----------------------------
ID_COL = "patient_id"
PHENO_COL = "phenotype"
AGE_COL = "age_group"

# For auto label mapping
EVENT_COL = "death_outcome"      # exists in death matrix
TIME_COL  = "survival_days"      # exists in death matrix

AGE_ALLOWED = {"<= 65 years", "> 65 years"}

# ----------------------------
# LOAD
# ----------------------------
df = pd.read_csv(IN_CSV)
df.columns = [c.strip() for c in df.columns]

needed = {ID_COL, PHENO_COL, AGE_COL}
missing = needed - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in {os.path.basename(IN_CSV)}: {missing}")

df[ID_COL] = df[ID_COL].astype(str).str.strip()

# strict age labels (strip only, no relabel)
df[AGE_COL] = df[AGE_COL].astype(str).str.strip()
bad_age = sorted(set(df[AGE_COL].dropna().unique()) - AGE_ALLOWED)
if bad_age:
    raise ValueError(f"Unexpected age_group labels found: {bad_age}. Expected: {sorted(AGE_ALLOWED)}")

# phenotype must be 0/1 numeric
df[PHENO_COL] = pd.to_numeric(df[PHENO_COL], errors="coerce")
df = df[df[PHENO_COL].isin([0, 1])].copy()
df[PHENO_COL] = df[PHENO_COL].astype(int)

# ----------------------------
# Robust event mapping (for label direction)
# ----------------------------
def map_event_to_01(series: pd.Series) -> pd.Series:
    raw = series.astype(str).str.strip()
    low = raw.str.lower()

    out = pd.Series(np.nan, index=series.index, dtype="float")

    # numeric direct
    num_try = pd.to_numeric(low.str.replace(",", "", regex=False), errors="coerce")
    out[num_try.isin([0, 1])] = num_try[num_try.isin([0, 1])]

    # string mapping (handles Present/Absent, Yes/No, Dead/Alive)
    out[out.isna() & low.str.contains(r"present|yes|dead|deceased|death|died|exitus", regex=True, na=False)] = 1
    out[out.isna() & low.str.contains(r"absent|no|alive|living|censor", regex=True, na=False)] = 0

    return pd.to_numeric(out, errors="coerce")

if EVENT_COL in df.columns:
    df["_event01"] = map_event_to_01(df[EVENT_COL])
else:
    df["_event01"] = np.nan

# ----------------------------
# Decide which phenotype is Frailty vs Resilient
# (higher death event rate => Accelerated Aging (Frailty))
# ----------------------------
if df["_event01"].notna().any():
    rates = df.groupby(PHENO_COL)["_event01"].mean().to_dict()
    if set(rates.keys()) == {0, 1} and all(pd.notna(list(rates.values()))):
        accelerated_ph = max(rates, key=rates.get)
    else:
        accelerated_ph = 1  # fallback if weird
else:
    accelerated_ph = 1  # fallback if event missing

decelerated_ph = 1 - accelerated_ph

PHENO_LABELS = {
    accelerated_ph: "Accelerated Aging (Frailty)",
    decelerated_ph: "Decelerated Aging (Resilient)",
}

df["phenotype_label"] = df[PHENO_COL].map(PHENO_LABELS)

print("Phenotype counts:\n", df["phenotype_label"].value_counts(dropna=False))
if df["_event01"].notna().any():
    print("Death event rate by phenotype (mapped 0/1):\n", df.groupby("phenotype_label")["_event01"].mean())

# ----------------------------
# Variable selection (AUTO)
# ----------------------------
# Exclude identifiers and outcome/leakage-ish columns from characterization
exclude_exact = {
    ID_COL, PHENO_COL, "phenotype_label",
    "_event01",
    # outcomes and direct survival constructs
    EVENT_COL, TIME_COL, "time", "event",
}

# also exclude obvious risk proxies used to build clusters if present
exclude_contains = [
    "risk_", "z_risk", "z_fi", "survival", "outcome"
]

def is_excluded(col: str) -> bool:
    if col in exclude_exact:
        return True
    low = col.lower()
    return any(key in low for key in exclude_contains)

candidate_cols = [c for c in df.columns if not is_excluded(c)]

# separate numeric and categorical by dtype and cardinality
numeric_cols = []
cat_cols = []

for c in candidate_cols:
    if pd.api.types.is_numeric_dtype(df[c]):
        numeric_cols.append(c)
    else:
        # try numeric coercion, but only treat as numeric if it actually becomes numeric with enough non-null
        coerced = pd.to_numeric(df[c], errors="coerce")
        non_null_ratio = coerced.notna().mean()
        unique_vals = df[c].nunique(dropna=True)

        # treat as numeric if mostly numeric and not just 0/1 disguised
        if non_null_ratio >= 0.80 and unique_vals > 2:
            df[c] = coerced
            numeric_cols.append(c)
        else:
            cat_cols.append(c)

# For categorical: keep reasonable cardinality (avoid free-text blowups like adr_description)
# You can adjust these thresholds.
MAX_LEVELS = 12
cat_cols = [c for c in cat_cols if df[c].nunique(dropna=True) <= MAX_LEVELS]

print(f"Candidate numeric predictors: {len(numeric_cols)}")
print(f"Candidate categorical predictors (<= {MAX_LEVELS} levels): {len(cat_cols)}")

# ----------------------------
# Build continuous summary: median [Q1, Q3] + missing
# ----------------------------
def summarize_continuous(g: pd.DataFrame, cols: list) -> pd.DataFrame:
    rows = []
    for c in cols:
        x = pd.to_numeric(g[c], errors="coerce")
        rows.append({
            "variable": c,
            "n_non_missing": int(x.notna().sum()),
            "missing": int(x.isna().sum()),
            "median": x.median(skipna=True),
            "q1": x.quantile(0.25),
            "q3": x.quantile(0.75),
        })
    out = pd.DataFrame(rows)
    out["summary"] = out.apply(
        lambda r: f"{r['median']:.2f} [{r['q1']:.2f}, {r['q3']:.2f}]" if pd.notna(r["median"]) else "NA",
        axis=1
    )
    return out[["variable", "summary", "n_non_missing", "missing"]]

cont_tables = []
for pheno, sub in df.groupby("phenotype_label"):
    tmp = summarize_continuous(sub, numeric_cols)
    tmp = tmp.rename(columns={
        "summary": f"{pheno} (median [Q1, Q3])",
        "missing": f"{pheno} missing",
        "n_non_missing": f"{pheno} n"
    })
    cont_tables.append(tmp)

if cont_tables:
    cont_merged = cont_tables[0]
    for t in cont_tables[1:]:
        cont_merged = cont_merged.merge(t, on="variable", how="outer")
else:
    cont_merged = pd.DataFrame(columns=["variable"])

cont_path = os.path.join(OUT_DIR, "clinical_characterization_continuous.csv")
cont_merged.to_csv(cont_path, index=False)

# ----------------------------
# Build categorical summary: count (%) per level + missing
# ----------------------------
def summarize_categorical(sub: pd.DataFrame, col: str) -> pd.DataFrame:
    s = sub[col].astype(str).str.strip()
    s = s.replace({"nan": np.nan, "None": np.nan, "": np.nan})
    counts = s.value_counts(dropna=False)
    n = len(s)

    out_rows = []
    for level, cnt in counts.items():
        if pd.isna(level):
            label = "Missing"
        else:
            label = str(level)
        pct = 100.0 * cnt / n if n > 0 else np.nan
        out_rows.append({"variable": col, "level": label, "count": int(cnt), "percent": pct})
    return pd.DataFrame(out_rows)

cat_all = []
for pheno, sub in df.groupby("phenotype_label"):
    for c in cat_cols:
        tmp = summarize_categorical(sub, c)
        tmp = tmp.rename(columns={
            "count": f"{pheno} count",
            "percent": f"{pheno} percent"
        })
        cat_all.append(tmp)

if cat_all:
    cat_df = pd.concat(cat_all, ignore_index=True)

    # Pivot to wide format with one row per (variable, level)
    keep_cols = ["variable", "level"]
    val_cols = [c for c in cat_df.columns if c not in keep_cols]
    cat_wide = cat_df.pivot_table(index=["variable", "level"], values=val_cols, aggfunc="first").reset_index()

    # Format percent columns to 1 decimal
    for c in cat_wide.columns:
        if c.endswith("percent"):
            cat_wide[c] = cat_wide[c].map(lambda x: round(x, 1) if pd.notna(x) else x)

else:
    cat_wide = pd.DataFrame(columns=["variable", "level"])

cat_path = os.path.join(OUT_DIR, "clinical_characterization_categorical.csv")
cat_wide.to_csv(cat_path, index=False)

# ----------------------------
# Optional: Export an Excel workbook (clean for clinicians)
# ----------------------------
xlsx_path = os.path.join(OUT_DIR, "clinical_characterization_table1.xlsx")
with pd.ExcelWriter(xlsx_path, engine="openpyxl") as writer:
    cont_merged.to_excel(writer, index=False, sheet_name="Continuous")
    cat_wide.to_excel(writer, index=False, sheet_name="Categorical")

print("\nSaved:")
print(" -", cont_path)
print(" -", cat_path)
print(" -", xlsx_path)


Phenotype counts:
 phenotype_label
Decelerated Aging (Resilient)    272
Accelerated Aging (Frailty)      134
Name: count, dtype: int64
Death event rate by phenotype (mapped 0/1):
 phenotype_label
Accelerated Aging (Frailty)      0.328358
Decelerated Aging (Resilient)    0.154412
Name: _event01, dtype: float64
Candidate numeric predictors: 33
Candidate categorical predictors (<= 12 levels): 38

Saved:
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_clean\clinical_characterization_continuous.csv
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_clean\clinical_characterization_categorical.csv
 - C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_clean\clinical_characterization_table1.xlsx


In [18]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

IN_CSV = r"C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_clean\clinical_characterization_categorical.csv"
OUT_DIR = r"C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_clean\categorical_plots"
os.makedirs(OUT_DIR, exist_ok=True)

PLOT_METRIC = "percent"   # or "count"
MAX_LEVELS_TO_PLOT = 15

# ----------------------------
# Load
# ----------------------------
df = pd.read_csv(IN_CSV)
df.columns = [c.strip() for c in df.columns]

# Identify phenotype columns
count_cols = [c for c in df.columns if c.lower().endswith("count")]
pct_cols   = [c for c in df.columns if c.lower().endswith("percent")]
value_cols = pct_cols if PLOT_METRIC == "percent" else count_cols

def pheno_name(col):
    return re.sub(r"\s+(count|percent)$", "", col, flags=re.I).strip()

phenotypes = [pheno_name(c) for c in value_cols]

# Regex for levels to exclude from plots
EXCLUDE_RE = re.compile(
    r"(missing|not\s*known|not\s*noted|unknown)",
    flags=re.IGNORECASE
)

def safe_name(s):
    s = re.sub(r"[<>:\"/\\|?*]", "_", str(s))
    s = re.sub(r"\s+", "_", s)
    return s[:160]

# ----------------------------
# Plot
# ----------------------------
for var in sorted(df["variable"].dropna().unique()):
    sub = df[df["variable"] == var].copy()

    # ---- DROP missing-style levels ----
    sub = sub[~sub["level"].astype(str).str.contains(EXCLUDE_RE, na=False)]

    if sub.empty:
        continue

    # Rank levels by total magnitude
    sub["_total"] = 0
    for c in value_cols:
        sub["_total"] += pd.to_numeric(sub[c], errors="coerce").fillna(0)

    sub = sub.sort_values("_total", ascending=False).drop(columns="_total")

    if len(sub) > MAX_LEVELS_TO_PLOT:
        sub = sub.iloc[:MAX_LEVELS_TO_PLOT]

    # Long format
    rows = []
    for c, ph in zip(value_cols, phenotypes):
        tmp = sub[["level", c]].copy()
        tmp["phenotype"] = ph
        tmp = tmp.rename(columns={c: "value"})
        tmp["value"] = pd.to_numeric(tmp["value"], errors="coerce").fillna(0)
        rows.append(tmp)

    plot_df = pd.concat(rows, ignore_index=True)

    wide = plot_df.pivot(
        index="level",
        columns="phenotype",
        values="value"
    ).fillna(0)

    ax = wide.plot(kind="bar", figsize=(11, 5))

    ax.set_title(f"{var} by Phenotype")
    ax.set_xlabel("Level")
    ax.set_ylabel("Percent (%)" if PLOT_METRIC == "percent" else "Count")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right")
    ax.grid(True, axis="y", alpha=0.2)

    plt.tight_layout()
    plt.savefig(
        os.path.join(OUT_DIR, f"{safe_name(var)}__{PLOT_METRIC}.png"),
        dpi=300
    )
    plt.close()

print("Plots saved to:", OUT_DIR)


C:\Users\HP\AppData\Local\Temp\ipykernel_1808\1132423419.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub["level"].astype(str).str.contains(EXCLUDE_RE, na=False)]
C:\Users\HP\AppData\Local\Temp\ipykernel_1808\1132423419.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub["level"].astype(str).str.contains(EXCLUDE_RE, na=False)]
C:\Users\HP\AppData\Local\Temp\ipykernel_1808\1132423419.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub["level"].astype(str).str.contains(EXCLUDE_RE, na=False)]
C:\Users\HP\AppData\Local\Temp\ipykernel_1808\1132423419.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str

C:\Users\HP\AppData\Local\Temp\ipykernel_1808\1132423419.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub["level"].astype(str).str.contains(EXCLUDE_RE, na=False)]
C:\Users\HP\AppData\Local\Temp\ipykernel_1808\1132423419.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub["level"].astype(str).str.contains(EXCLUDE_RE, na=False)]
C:\Users\HP\AppData\Local\Temp\ipykernel_1808\1132423419.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub["level"].astype(str).str.contains(EXCLUDE_RE, na=False)]
C:\Users\HP\AppData\Local\Temp\ipykernel_1808\1132423419.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str

Plots saved to: C:\Users\HP\OneDrive\Desktop\Phase 3\phenotypes_clean\categorical_plots
